In [1]:
#General packages
import numpy as np
from My_Jampy import JAM
import emcee

#MPI
from schwimmbad import MPIPool

#Constants and usefull packages
from astropy.cosmology import Planck15 as cosmo
from astropy.constants import G, M_sun, c
import astropy.units as u

#Autolens Model packages
import autolens as al
import autolens.plot as aplt

#Combined Model package
import CombinedModel

#Useful constants
metre2Mpc = (1*u.m).to(u.Mpc)/u.m           #Constant factor to convert metre to Mpc.
kg2Msun = (1*u.kg/M_sun)*u.solMass/u.kg     #Constant factor to convert kg to Msun

G_Mpc = G*(metre2Mpc)**3/kg2Msun            #Gravitational constant in Mpc³/(Msun s²)
c_Mpc = c*metre2Mpc                         #Speed of light in Mpc/s


#Dataset path
data_folder = "/home/carlos/Documents/GitHub/Master-Degree/Autolens_tests/autolens_workspace/Test_3/Simulation_data/"

In [2]:
#Reading MGE inputs
            #attention to units
surf_lum, sigma_lum, qobs_lum = np.loadtxt("JAM_Input.txt", unpack=True)          #MGE decomposition
surf_dm, sigma_dm , qobs_dm   = np.loadtxt("SDP81_pseudo-DM.txt", unpack=True)    #DM component
norm_psf, sigma_psf           = np.loadtxt("MUSE_Psf_model.txt", unpack=True)     #PSF
xbin, ybin, vrms              = np.loadtxt("vrms_data.txt", unpack=True)          #Vrms data

muse_pixsize = 0.2                            #Muse pixel size [arcsec/px]

z_lens   = 0.299                                    #Lens redshifth
z_source = 3.100                                    #Source redshift

#Angular diameter distances
D_l = cosmo.angular_diameter_distance(z_lens)                   #Lens              
D_s = cosmo.angular_diameter_distance(z_source)                 #Source
D_ls = cosmo.angular_diameter_distance_z1z2(z_lens, z_source)   #Lens to source

## Models inicialization

"""
    To inicialize the model, we set some random values for the parameters. But it's only necessary for initialize the model. During the non-linear search, this values will be updated constantly until the best fit.
"""  


   #This quantities are our unknown parameters
inc       = 75                              #Inclination [deg]
mbh       = 1e10                            #Mass of black hole [M_sun]
beta      = np.full_like(surf_lum, 0.3)     #Anisotropy
ml        = 10                              #Mass to light ratio [M_sun/L_sun]
mag_shear = 0.01                            #Shear magnitude
phi_shear = 100.0                             #Shear angle
rho_s     = 1e10                            #dark matter intensity
qdm       = np.full_like(qobs_dm, 0.5)      #dark matter axial ratio
gamma     = 1.0                             #Gamma

#--------------------------------------------------------------------------------------------------#
# JAMPY MODEL

Jam_model = JAM(ybin=ybin, xbin=xbin, inc=inc, distance=D_l.value, mbh=mbh, beta=beta, rms=vrms,
               normpsf=norm_psf, sigmapsf=sigma_psf*muse_pixsize, pixsize=muse_pixsize)

    #Add Luminosity component
Jam_model.luminosity_component(surf_lum=surf_lum, sigma_lum=sigma_lum,
                                qobs_lum=qobs_lum, ml=ml)
    #Add DM component
Jam_model.DM_component(surf_dm=rho_s * surf_dm, sigma_dm=sigma_dm, qobs_dm=qdm)


#--------------------------------------------------------------------------------------------------#
# PYAUTOLENS MODEL

imaging = al.Imaging.from_fits(
    image_path=f"{data_folder}/arcs_simulation.fits",
    noise_map_path=f"{data_folder}/noise_simulation.fits",
    psf_path=f"{data_folder}/psf_simulation.fits",
    pixel_scales=0.1,
)

mask = al.Mask.circular_annular(centre=(0.0, -0.2), inner_radius=1., outer_radius=2.3,
                          pixel_scales=imaging.pixel_scales, shape_2d=imaging.shape_2d) #Create a mask

masked_image = al.MaskedImaging(imaging=imaging, mask=mask, inversion_uses_border=True) #Masked image

mass_profile = al.mp.MGE()

    #Components
mass_profile.MGE_comps(z_l=z_lens, z_s=z_source, 
                   surf_lum=surf_lum, sigma_lum=sigma_lum, qobs_lum=qobs_lum, ml=ml,
                   mbh=mbh, surf_dm =rho_s * surf_dm, sigma_dm=sigma_dm, qobs_dm=qdm)

#--------------------------------------------------------------------------------------------------#
# COMBINED MODEL

    #Just remembering, by default the model does not include dark matter.
model = CombinedModel.Models(Jampy_model=Jam_model, mass_profile=mass_profile,
                             masked_imaging=masked_image, quiet=True)

model.mass_to_light(ml_kind='scalar')               #Setting gradient ML
model.beta(beta_kind='scalar')                      #Seting vector anisotropy
model.has_DM(a=True,filename="SDP81_pseudo-DM.txt") #Setting Dark matter component
#--------------------------------------------------------------------------------------------------#

In [66]:
def call_model(x0):
    #print(x0)
    value =  model(x0)
    #print(value)
    if np.isfinite(value):
        return -value
    else:
        return 1e200

In [69]:
from scipy.optimize import differential_evolution

In [70]:
#In order: ML, beta, inc, log_mbh, log_rho_s, qDM, mag_shear, phi_shear, gamma
bounds = [(1, 12), (-2, 2), (40, 90), (6, 12), (6, 12), (0.2, 1), (0, 0.1), (0, 179), (0.8, 1.2)]

In [72]:
result = differential_evolution(call_model, bounds=bounds, updating='deferred', disp=True)

[ 7.70629744e+00 -7.82915599e-01  7.17116226e+01  7.64689101e+00
  8.82064184e+00  3.66679816e-01  4.55318598e-02  9.11017296e+01
  8.78136580e-01]
-23157.573859292137
[ 7.63150127e+00 -9.81598463e-01  7.75437628e+01  7.62137010e+00
  8.96448164e+00  5.64982511e-01  2.93014476e-02  8.53471648e+01
  1.06931090e+00]
-8191.9155961449715
[ 7.12624745e+00 -8.22610634e-01  7.15251983e+01  8.05457809e+00
  8.10739195e+00  5.84487524e-01  2.26911120e-02  8.62717088e+01
  1.14233768e+00]
-7030.248010855758
[ 7.84530754e+00 -8.12428606e-01  7.12721376e+01  7.53311205e+00
  8.39240448e+00  8.44758639e-01  3.18069196e-02  8.83375512e+01
  9.92546671e-01]
-7538.092382957611
[ 7.36495901e+00 -2.07121519e-01  7.15870001e+01  8.23686578e+00
  8.82547712e+00  9.60419545e-01  4.98826827e-02  8.85972540e+01
  1.05244169e+00]
-11923.622267826591
[ 7.69274610e+00 -1.40715746e-01  7.72094720e+01  7.54967022e+00
  8.02187744e+00  4.98781334e-01  1.11667150e-02  9.25716097e+01
  1.04053654e+00]
-8902.06986284

-14996.28681809296
[ 7.46994150e+00 -7.73598458e-01  7.53933025e+01  8.13176246e+00
  8.29267124e+00  5.32149583e-01  4.80474951e-02  8.11996866e+01
  8.09437818e-01]
-62139.17182473062
[ 7.57249363e+00 -8.64584891e-01  7.65241890e+01  8.28501763e+00
  8.65611144e+00  3.18951301e-01  1.66763500e-02  9.43191316e+01
  8.70923349e-01]
-35696.49840245475
[ 7.79802819e+00 -7.24817374e-01  7.50554668e+01  7.93651636e+00
  8.92628027e+00  2.72123096e-01  1.28679540e-02  9.01390110e+01
  9.77038308e-01]
-6306.399286723373
[ 7.07805212e+00 -2.87187927e-01  7.58386213e+01  7.56709422e+00
  8.69627496e+00  6.70083040e-01  3.50069141e-02  8.50344226e+01
  9.60579473e-01]
-46455.70056920887
[ 7.86309997e+00 -6.16626640e-01  7.14035460e+01  7.74587637e+00
  8.85670698e+00  4.73537500e-01  3.42638084e-03  9.33422591e+01
  1.02193146e+00]
-11115.240534616125
[ 7.83635172e+00 -9.86639446e-01  7.87114969e+01  7.67266417e+00
  8.21924912e+00  6.24510340e-01  4.00039780e-02  9.19539415e+01
  8.28592774e-0

-55475.75862884277
[ 7.21442570e+00 -4.40936811e-01  7.27950871e+01  7.63539694e+00
  8.60695124e+00  6.10046500e-01  4.27356928e-02  9.28182809e+01
  1.12923618e+00]
-8392.960772618915
[ 7.93423516e+00 -9.74995270e-01  7.06827712e+01  7.99112419e+00
  8.42998598e+00  7.84415689e-01  2.67004244e-02  8.74954716e+01
  8.17440631e-01]
-25604.9363396794
[ 7.17621488e+00 -9.36294664e-02  7.09296379e+01  7.52120562e+00
  8.78118861e+00  6.49409432e-01  2.30459877e-02  9.00315509e+01
  1.00042664e+00]
-29683.312804407506
[ 7.40285012e+00 -2.21117627e-01  7.47731284e+01  8.30258681e+00
  8.89829977e+00  8.36372737e-01  4.87629779e-02  9.08833361e+01
  1.06649133e+00]
-9768.7359172672
[ 7.55667029e+00 -3.29150731e-01  7.62747598e+01  8.14626963e+00
  8.86334787e+00  4.43598782e-01  1.30019240e-02  8.98326395e+01
  9.04029549e-01]
-28428.434719750065
[ 7.34616365e+00 -9.22065621e-01  7.80475866e+01  8.36463694e+00
  8.13294780e+00  8.55422185e-01  4.13593059e-02  8.51353617e+01
  9.89413086e-01]

-14794.280648500982
[ 7.59954729e+00 -9.75580652e-01  7.23261880e+01  8.10775934e+00
  8.99669876e+00  2.88320777e-01  3.29201349e-02  8.71399998e+01
  1.11900049e+00]
-inf
[ 7.89033374e+00 -2.74806400e-01  7.40510119e+01  8.23769262e+00
  8.16901072e+00  3.34336166e-01  2.75308020e-02  8.38536449e+01
  1.04474901e+00]
-14449.69338034583
[ 7.03378605e+00 -7.93655782e-01  7.67681302e+01  8.01295097e+00
  8.97938047e+00  2.64637130e-01  2.76819489e-02  9.41261065e+01
  1.10982007e+00]
-13209.677828650576
[ 7.36299993e+00 -4.79799326e-01  7.17870953e+01  7.59965302e+00
  8.81883144e+00  5.60221056e-01  1.40286490e-02  8.76124668e+01
  1.16978642e+00]
-10906.141717638044
[ 7.62477348e+00 -7.58084770e-01  7.42308645e+01  8.36880496e+00
  8.39299896e+00  4.40012540e-01  1.84060029e-02  9.23210272e+01
  1.12530601e+00]
-17411.036183016273
[ 7.42095919e+00 -2.69707120e-01  7.20878566e+01  7.72338746e+00
  8.20327455e+00  7.99646471e-01  1.41893131e-02  8.69092790e+01
  1.16154840e+00]
-12920.9

-15625.933560039333
[ 7.32123892e+00 -6.10325166e-01  7.60356420e+01  8.17443421e+00
  8.66801593e+00  3.59093063e-01  3.99145883e-02  8.63916340e+01
  1.15388889e+00]
-9845.111394710737
[ 7.45731109e+00 -9.94181866e-01  7.61299076e+01  7.81201157e+00
  8.83027547e+00  5.36081769e-01  7.37219951e-03  8.19988379e+01
  1.05768826e+00]
-7445.537773385039
[ 7.61972463e+00 -1.06843772e-01  7.01517366e+01  7.86519487e+00
  8.73889047e+00  2.54047805e-01  3.24220070e-02  8.86433605e+01
  9.12746193e-01]
-inf
[ 7.22453275e+00 -3.55463420e-01  7.51585977e+01  7.86410268e+00
  8.74397789e+00  2.77553598e-01  4.20866675e-02  9.26738274e+01
  9.91635973e-01]
-26112.205455695424
[ 7.61898623e+00 -6.78971723e-01  7.53279126e+01  8.03059082e+00
  8.77110698e+00  2.10877188e-01  2.59975408e-03  8.35075344e+01
  8.89992576e-01]
-inf
[ 7.13492637e+00 -9.33843010e-01  7.64635385e+01  8.31099960e+00
  8.77597663e+00  6.39012344e-01  1.59326646e-02  8.77838557e+01
  1.11016677e+00]
-9236.510140687451
[ 7.7

...

-6366.9871866601525
[ 7.66274133e+00 -7.00629947e-01  7.25936751e+01  7.78157720e+00
  8.96206703e+00  6.24059237e-01  1.52487808e-02  8.62989467e+01
  1.09573407e+00]
-13080.774247690466
[ 7.42365643e+00 -1.93613887e-01  7.23477688e+01  8.13774342e+00
  8.28362818e+00  7.06848774e-01  5.74523980e-03  9.07446970e+01
  1.10865677e+00]
-9043.482552767107
[ 7.90750300e+00 -9.19254390e-01  7.21971851e+01  8.04877099e+00
  8.98463347e+00  3.32089579e-01  4.51321279e-02  8.65919828e+01
  1.17373972e+00]
-50126.1802304003
[ 7.14683128e+00 -9.21146792e-01  7.16088079e+01  7.89351551e+00
  8.97634575e+00  9.36505696e-01  1.94565682e-02  8.37970798e+01
  1.16744241e+00]
-6137.8012456028555
[ 7.10867254e+00 -4.73891885e-01  7.41119905e+01  7.63867645e+00
  8.50494986e+00  3.66679816e-01  8.42041586e-03  8.58082048e+01
  9.26919994e-01]
-67691.35351206508
[ 7.40799890e+00 -1.80827720e-01  7.18538826e+01  7.99286794e+00
  8.79711186e+00  7.05820430e-01  2.06253571e-02  9.32420254e+01
  9.12269792e-

-48006.63626098236
[ 7.70236060e+00 -9.79694352e-01  7.79452700e+01  8.18113638e+00
  8.21150905e+00  4.54100026e-01  3.77248588e-03  9.03338362e+01
  1.05031843e+00]
-9481.641298325585
[ 7.27009295e+00 -7.21045335e-01  7.62722110e+01  7.82026225e+00
  8.67613935e+00  5.46237880e-01  3.16071101e-02  8.90372990e+01
  1.16102338e+00]
-8107.720805807832
[ 7.97812085e+00 -7.81044640e-01  7.73923780e+01  8.46088643e+00
  8.60676954e+00  8.76194612e-01  1.44948574e-03  8.30960911e+01
  1.16736354e+00]
-58099.1064866571
differential_evolution step 4: f(x)= 6111.37
[ 7.07554588e+00 -2.72725879e-01  7.49804665e+01  7.94031999e+00
  8.77300952e+00  2.80297642e-01  3.07260409e-02  8.29525311e+01
  8.45029594e-01]
-81626.14348957781
[ 7.33177864e+00 -3.56946952e-01  7.78783380e+01  7.72920424e+00
  8.96448164e+00  5.64982511e-01  2.41512908e-02  8.73407608e+01
  1.10218983e+00]
-6508.806916502582
[ 7.73798436e+00 -8.33589248e-01  7.99370437e+01  8.21324289e+00
  8.68439352e+00  4.40785248e-01  2.0

-8320.25183714422
[ 7.00648897e+00 -7.09399166e-01  7.61753529e+01  7.79978300e+00
  8.79467075e+00  6.87284629e-01  7.26504422e-03  8.46347232e+01
  9.89895661e-01]
-47583.77471957654
[ 7.07972231e+00 -2.59127077e-01  7.51425024e+01  7.58125015e+00
  8.37608248e+00  9.85456923e-01  1.99244004e-02  8.43016952e+01
  1.14600274e+00]
-8572.760024307907
[ 7.23210466e+00 -5.91616865e-01  7.59517937e+01  7.75326396e+00
  8.68983481e+00  6.01758052e-01  1.27203550e-02  8.95024721e+01
  1.07302708e+00]
-11181.851540354975
[ 7.06172314e+00 -8.09502905e-01  7.57257324e+01  7.68771115e+00
  8.81463962e+00  4.68495416e-01  1.40465974e-02  8.64541745e+01
  1.12553770e+00]
-10650.353520265022
[ 7.19645639e+00 -7.73598458e-01  7.41273541e+01  7.93879509e+00
  8.60351252e+00  7.42764232e-01  2.57347072e-02  9.07714056e+01
  1.09772097e+00]
-8529.852231064979
[ 7.21440502e+00 -8.11809349e-01  7.44719634e+01  7.72392216e+00
  8.76089525e+00  6.40054693e-01  3.01523182e-02  8.57118438e+01
  1.15032613e+0

-11570.356133888317
[ 7.37764837e+00 -9.39936276e-01  7.62676596e+01  8.30354719e+00
  8.02988692e+00  6.29578997e-01  2.25608512e-02  8.29775004e+01
  1.07615146e+00]
-6645.507948492132
[ 7.42723096e+00 -9.46925888e-01  7.53143398e+01  7.54572739e+00
  8.50700323e+00  8.88035470e-01  1.87837553e-02  8.94156746e+01
  9.96433907e-01]
-14555.407240823253
[ 7.16090796e+00 -9.02375157e-01  7.34836523e+01  7.79790251e+00
  8.72986023e+00  3.72202045e-01  2.22643383e-02  9.46193521e+01
  1.05576438e+00]
-16548.540316549857
[ 7.14560697e+00 -7.87474682e-01  7.58194775e+01  8.19647382e+00
  8.84407867e+00  9.14616985e-01  2.02178937e-02  9.21309413e+01
  1.17661661e+00]
-6787.103983263149
[ 7.38529332e+00 -7.98262199e-01  7.43073586e+01  8.00277566e+00
  8.86269610e+00  6.10046500e-01  1.26967931e-02  8.53861673e+01
  1.04242701e+00]
-9396.144802869147
[ 7.22734058e+00 -8.59341488e-01  7.41900021e+01  7.99112419e+00
  8.62176727e+00  7.84415689e-01  2.67004244e-02  9.46077347e+01
  1.03791167e

-6091.0807066119305
[ 7.56201276e+00 -9.26282057e-01  7.26452164e+01  7.73556097e+00
  8.84472439e+00  4.43372606e-01  2.05998298e-02  8.62777707e+01
  1.02898692e+00]
-6101.545483132808
[ 7.56201275e+00 -9.26282047e-01  7.26452164e+01  7.73556097e+00
  8.84472439e+00  4.43372606e-01  2.05998298e-02  8.62777707e+01
  1.02898692e+00]
-6101.178023302713
[ 7.56201275e+00 -9.26282057e-01  7.26452164e+01  7.73556097e+00
  8.84472439e+00  4.43372606e-01  2.05998298e-02  8.62777707e+01
  1.02898692e+00]
-6097.13096515768
[ 7.56201275e+00 -9.26282057e-01  7.26452164e+01  7.73556098e+00
  8.84472439e+00  4.43372606e-01  2.05998298e-02  8.62777707e+01
  1.02898692e+00]
-6087.379898137218
[ 7.56201275e+00 -9.26282057e-01  7.26452164e+01  7.73556097e+00
  8.84472440e+00  4.43372606e-01  2.05998298e-02  8.62777707e+01
  1.02898692e+00]
-6085.369616066447
[ 7.56201275e+00 -9.26282057e-01  7.26452164e+01  7.73556097e+00
  8.84472439e+00  4.43372616e-01  2.05998298e-02  8.62777707e+01
  1.02898692e+00

In [73]:
print(result)

array([ 7.56201275e+00, -9.26282057e-01,  7.26452164e+01,  7.73556097e+00,
        8.84472439e+00,  4.43372606e-01,  2.05998298e-02,  8.62777707e+01,
        1.02898692e+00])

## Model Plot

model_plot = CombinedModel.Models(Jampy_model=Jam_model, mass_profile=mass_profile,
                             masked_imaging=masked_image, quiet=False)

model_plot.mass_to_light(ml_kind='scalar')               #Setting gradient ML
model_plot.beta(beta_kind='scalar')                      #Seting vector anisotropy
model_plot.has_DM(a=True,filename="SDP81_pseudo-DM.txt") #Setting Dark matter component
#--------------------------------------------------------------------------------------------------#

model_plot(result.x)